In [2]:
!pip install pandas-ods-reader

     |████████████████████████████████| 125 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 5.1 MB/s eta 0:00:01
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49003 sha256=79017fb046a49695cc6ae7b1e7fabb5af2cd3577c25c834ec3e64e216d6e9cda
  Stored in directory: /home/mill/.cache/pip/wheels/50/d3/91/67b352fa0847576d7f3716a1d9a8ded88abb49e6c972cec44a
Successfully built ezodf
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3


In [17]:
from pandas_ods_reader import read_ods
import pandas as pd

In [40]:
df = read_ods("australian_2021population.ods")

In [41]:
# Filter age groups from 20 to 49
df = df[df['Age group (years)'].isin(['20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49'])]

# Calculate total percentage of the selected age groups
total_percentage = df['%'].sum()

# Rescale the percentages to sum to 100%
df['Rescaled %'] = (df['%'] / total_percentage) * 100


In [42]:
# Function to split age group into individual ages and distribute the Rescaled %
def split_age_group(row):
    start_age, end_age = map(int, row['Age group (years)'].split(' to '))
    if end_age == 49:
        end_age += 1  # Include age 50 in the 45 to 49 age group
    num_ages = end_age - start_age + 1
    rescaled_percentage_per_age = row['Rescaled %'] / num_ages
    return [(age, rescaled_percentage_per_age) for age in range(start_age, end_age + 1)]

# Split age groups into individual ages
individual_ages = []
for _, row in df.iterrows():
    individual_ages.extend(split_age_group(row))

# Create a new DataFrame for individual ages
individual_age_df = pd.DataFrame(individual_ages, columns=['Age', 'Rescaled %'])

# Adjust the final value to ensure the sum is 100%
total_rescaled_percentage = individual_age_df["Rescaled %"].sum()
difference = 100 - total_rescaled_percentage
individual_age_df.loc[individual_age_df.index[-1], 'Rescaled %'] += difference


In [44]:
individual_age_df.to_csv("australian_2021rescale%.csv")